In [109]:
import duckdb

In [2]:
con = duckdb.connect(database = "csecicids2018.duckdb", read_only = False)

In [3]:
con.execute('CREATE OR REPLACE TABLE flows AS SELECT "Flow ID", Label FROM read_csv_auto("../datasets/CSECICIDS2018_improved/*.csv");')

In [4]:
con.execute("UPDATE Flows SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [5]:
con.execute('SELECT DISTINCT Label FROM flows')
Labels = con.fetch_df()
Labels

,Label
0,DDoS-HOIC
1,Web Attack - XSS
2,Infiltration - NMAP Portscan
3,Web Attack - Brute Force
4,BENIGN
5,DDoS-LOIC-UDP
6,Botnet Ares
7,Infiltration - Communication Victim Attacker
8,DoS Slowloris
9,DoS GoldenEye


In [6]:
con.execute('SELECT Label, COUNT(*) AS count FROM Flows GROUP BY Label')
counted_labels = con.fetch_df()
counted_labels

,Label,count
0,DDoS-HOIC,1082293
1,Web Attack - XSS,113
2,Infiltration - NMAP Portscan,89374
3,Web Attack - Brute Force,131
4,BENIGN,59659723
5,Infiltration - Communication Victim Attacker,204
6,Botnet Ares,142921
7,DDoS-LOIC-UDP,2527
8,DoS Slowloris,8490
9,SSH-BruteForce,94197


In [85]:
con.execute('CREATE OR REPLACE TABLE sampled_flows ("Flow ID" VARCHAR, Label VARCHAR)')

In [87]:
for label in Labels.to_numpy():
    con.execute('WITH tab1 AS (SELECT "Flow ID", Label FROM flows WHERE Label = \'{}\' ORDER BY RANDOM() LIMIT 2000) INSERT INTO sampled_flows SELECT * FROM tab1'.format(label[0]))

In [88]:
con.execute('SELECT Label, COUNT(*) AS count FROM sampled_flows GROUP BY Label')
con.fetchall()

[('DDoS-HOIC', 2000),
 ('Web Attack - XSS', 113),
 ('Infiltration - NMAP Portscan', 2000),
 ('Web Attack - Brute Force', 131),
 ('BENIGN', 2000),
 ('DDoS-LOIC-UDP', 2000),
 ('Botnet Ares', 2000),
 ('Infiltration - Communication Victim Attacker', 204),
 ('DoS Slowloris', 2000),
 ('DoS GoldenEye', 2000),
 ('SSH-BruteForce', 2000),
 ('DoS Hulk', 2000),
 ('Infiltration - Dropbox Download', 85),
 ('DDoS-LOIC-HTTP', 2000),
 ('Web Attack - SQL', 39)]

In [103]:
con.execute('CREATE OR REPLACE TABLE CSECICIDS2018 ("Protocol" BIGINT, "Flow Duration" BIGINT, "Total Fwd Packet" BIGINT, "Total Bwd packets" BIGINT, "Total Length of Fwd Packet" BIGINT, "Total Length of Bwd Packet" BIGINT, "Fwd Packet Length Max" BIGINT, "Fwd Packet Length Min" BIGINT, "Fwd Packet Length Mean" DOUBLE, "Fwd Packet Length Std" DOUBLE, "Bwd Packet Length Max" BIGINT, "Bwd Packet Length Min" BIGINT, "Bwd Packet Length Mean" DOUBLE, "Bwd Packet Length Std" DOUBLE, "Flow Bytes/s" DOUBLE, "Flow Packets/s" DOUBLE, "Flow IAT Mean" DOUBLE, "Flow IAT Std" DOUBLE, "Flow IAT Max" BIGINT, "Flow IAT Min" BIGINT, "Fwd IAT Total" BIGINT, "Fwd IAT Mean" DOUBLE, "Fwd IAT Std" DOUBLE, "Fwd IAT Max" BIGINT, "Fwd IAT Min" BIGINT, "Bwd IAT Total" BIGINT, "Bwd IAT Mean" DOUBLE, "Bwd IAT Std" DOUBLE, "Bwd IAT Max" BIGINT, "Bwd IAT Min" BIGINT, "Fwd PSH Flags" BIGINT, "Bwd PSH Flags" BIGINT, "Fwd URG Flags" BIGINT, "Bwd URG Flags" BIGINT, "Fwd RST Flags" BIGINT, "Bwd RST Flags" BIGINT, "Fwd Header Length" BIGINT, "Bwd Header Length" BIGINT, "Fwd Packets/s" DOUBLE, "Bwd Packets/s" DOUBLE, "Packet Length Min" BIGINT, "Packet Length Max" BIGINT, "Packet Length Mean" DOUBLE, "Packet Length Std" DOUBLE, "Packet Length Variance" DOUBLE, "FIN Flag Count" BIGINT, "SYN Flag Count" BIGINT, "RST Flag Count" BIGINT, "PSH Flag Count" BIGINT, "ACK Flag Count" BIGINT, "URG Flag Count" BIGINT, "CWR Flag Count" BIGINT, "ECE Flag Count" BIGINT, "Down/Up Ratio" DOUBLE, "Average Packet Size" DOUBLE, "Fwd Segment Size Avg" DOUBLE, "Bwd Segment Size Avg" DOUBLE, "Fwd Bytes/Bulk Avg" BIGINT, "Fwd Packet/Bulk Avg" BIGINT, "Fwd Bulk Rate Avg" BIGINT, "Bwd Bytes/Bulk Avg" BIGINT, "Bwd Packet/Bulk Avg" BIGINT, "Bwd Bulk Rate Avg" BIGINT, "Subflow Fwd Packets" BIGINT, "Subflow Fwd Bytes" BIGINT, "Subflow Bwd Packets" BIGINT, "Subflow Bwd Bytes" BIGINT, "FWD Init Win Bytes" BIGINT, "Bwd Init Win Bytes" BIGINT, "Fwd Act Data Pkts" BIGINT, "Fwd Seg Size Min" BIGINT, "Active Mean" DOUBLE, "Active Std" DOUBLE, "Active Max" BIGINT, "Active Min" BIGINT, "Idle Mean" DOUBLE, "Idle Std" DOUBLE, "Idle Max" BIGINT, "Idle Min" BIGINT, "ICMP Code" BIGINT, "ICMP Type" BIGINT, "Total TCP Flow Time" BIGINT, "Label" VARCHAR(255), "Attempted Category" BIGINT )')

In [91]:
con.execute('SELECT "Flow ID" FROM sampled_flows') 
sampled_flows = con.fetchdf()

In [98]:
for flow in sampled_flows.to_numpy():
    con.execute('INSERT INTO CSECICIDS2018 SELECT * EXCLUDE (id,"Flow ID", "Src IP","Src Port","Dst IP","Dst Port",Timestamp) FROM read_csv_auto("../datasets/CSECICIDS2018_improved/*.csv") WHERE "Flow ID" = \'{}\''.format(flow[0]))

RuntimeError: Query interrupted

In [71]:
con.execute("UPDATE CSECICIDS2018_SAMPLED SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [58]:
con.execute("COPY (SELECT * FROM CSECICIDS2018_SAMPLED) TO '../datasets/CSECICIDS2018_improved.parquet' (FORMAT PARQUET)")